# Devoir 1, Question 3 : Classement avec option de rejet

# Homework 1, Question 3: Classification with reject option

Soit un classifieur bayésien de loi normale multivariée avec matrices de covariance distinctes pour chaque classe et isotropiques, c’est-à-dire avec des valeurs égales sur toute la diagonale et nulles autrement, $\mathbf{\Sigma}_i=\sigma_i^2\mathbf{I},~\forall i$.

Consider a Bayesian multivariate Gaussian distribution classifier with distinct covariance matrices for each class and isotropic, that is, with values equal across the diagonal and zero otherwise, $\mathbf{\Sigma}_i=\sigma_i^2\mathbf{I},~\forall i$.

## Librairies permises

## Allowed libraries

In [ ]:
%matplotlib notebook
import time
import numpy
import pandas
import collections

from IPython import display

import matplotlib
matplotlib.rcParams['figure.figsize'] = (9.0, 7.0)
from matplotlib import pyplot

# Jeux de données / datasets
from sklearn.datasets import load_iris

# Méthodes d'évaluation / evaluation methods
from sklearn.model_selection import train_test_split, RepeatedKFold

## Q3A
Calculez l’équation pour l’estimation du paramètre $\sigma_{i}$ par la méthode du maximum de vraisemblance, en fournissant les développements mathématiques complets dans votre rapport.

## Q3A
Calculate the equation for the estimation of the parameter $\sigma_{i}$ by the maximum likelihood method, providing the full mathematical developments in your report.

### Entrez votre solution à Q3A dans la cellule ci-dessous (markdown et $\LaTeX$)

### Enter your answer to Q3A in the cell below (markdown and $\LaTeX$)

<div class="feedback-cell" style="background: rgba(100 , 100 , 100 , 0.4)">
            <h3>Votre soumission a été enregistrée!</h3><ul><li>notez qu'il n'y a
            <strong>pas</strong> de correction automatique pour cet exercice&puncsp;;</li>
            <li>par conséquent, votre note est <strong>actuellement</strong> zéro&puncsp;;</li>
            <li>elle sera cependant ajustée par le professeur dès que la correction manuelle
            sera complétée&puncsp;;</li><li>vous pouvez soumettre autant de fois que nécessaire
            jusqu'à la date d'échéance&puncsp;;</li><li>mais évitez de soumettre inutilement.</li>
            </ul></div>

Soit la loi normale multivariée: \
$p(x)= \frac{1}{(2\pi)^{0.5D}|\Sigma|^{0.5}}\exp(-\frac{1}{2}(x-\mu)^T\Sigma^{-1}(x-\mu))$

$p(x_i) \sim \mathcal{N}_D(\mu_i, \sigma_i^2)$
avec $\Sigma_i = \sigma_i^2I, \forall i$ 

Fonction de log-vraissemble:                
$\begin{align*}
L(\mu_i,\sigma_i^2|\mathcal{X}) &= \log\prod\limits_{t=1}^{N} p(x_i^t|\mu_i,\sigma_i^2) \\
                       &= \sum\limits_{t=1}^{N} \log\frac{1}{(2\pi)^{0.5D}|\Sigma_i|^{0.5}}\exp\big(-\frac{1}{2}(x_i^t-\mu_i)^T\Sigma_i^{-1}(x_i^t-\mu_i)\big) \\
                       &\equiv -\frac{ND}{2}\log2\pi -\frac{N}{2}log|\Sigma_i| -\frac{1}{2}\sum\limits_{t=1}^{N} (x_i^t-\mu_i)^T\Sigma_i^{-1}(x_i^t-\mu_i) \\
                       &\equiv -\frac{N}{2}log|\sigma_i^2I| -\frac{1}{2}\sum\limits_{t=1}^{N} (x_i^t-\mu_i)^T(\sigma_i^2I)^{-1}(x_i^t-\mu_i) \\
                       &\equiv -Nlog\sigma_i -\frac{1}{2\sigma_i^2}\sum\limits_{t=1}^{N} (x_i^t-\mu_i)^T(x_i^t-\mu_i) \\
\end{align*}$

Estimation du maximum de vraissemblance: \
$\frac{\partial L(\mu_i,\sigma_i^2|\mathcal{X}) }{\partial\mu_i} \equiv \sum\limits_{t=1}^{N} x_i^t - N\mu_i  \\
\frac{\partial L(\mu_i,\sigma_i^2|\mathcal{X}) }{\partial\mu_i} = 0 
\implies \sum\limits_{t=1}^{N} x_i^t - Nm_i = 0 
\implies m_i = \frac{\sum\limits_{t=1}^{N} x_i^t}{N}$
$\frac{\partial L(\mu_i,\sigma_i^2|\mathcal{X}) }{\partial\sigma_i} \equiv -\frac{N}{\sigma_i} + \frac{1}{\sigma_i^3}\sum\limits_{t=1}^{N} (x_i^t-\mu_i)^2 \\
\frac{\partial L(\mu_i,\sigma_i^2|\mathcal{X}) }{\partial\sigma_i} = 0 
\implies -\frac{N}{s_i} + \frac{1}{s_i^3}\sum\limits_{t=1}^{N} (x_i^t - m_i)^2 = 0 
\implies s^2 = \frac{\sum\limits_{t=1}^{N} (x_i^t-m_i)^2}{N}$

## Q3B
Supposons maintenant que l'on ajoute une option de rejet au classement. Le coût des erreurs est égal pour tous les  types d'erreurs (coût de 1), sauf pour le rejet (coût de $\lambda \in [0,1]$). Donnez l'équation complète pour calculer la probabilité a posteriori $P(C_{i}\mid \mathbf{x})$ et la fonction pour la prise de décision minimisant le risque (minimisant le coût) avec l'option de rejet.

## Q3B
Now suppose we add a reject option to the ranking. The cost of errors is equal for all types of errors (cost of 1), except for rejection (cost of $\lambda \in [0,1]$). Give the full equation to calculate the posterior probability $P(C_{i}\mid \mathbf{x})$ and the function for decision-making minimizing risk (minimizing cost) with the reject option.

### Entrez votre solution à Q3B dans la cellule ci-dessous (markdown et $\LaTeX$)

### Enter your answer to Q3B in the cell below (markdown and $\LaTeX$)

La fonction de perte: 
$\mathcal{L}(\alpha_i, C_j) = 
\begin{cases}
0 & \textrm{ si } i = j \\
\lambda & \textrm{ si } i = K+1 \\
1 & \textrm{ autrement }
\end{cases}$

La fonction discriminante associée est: \
$\begin{align*}
P(C_i|x) = h_i(x) &= -\frac{||x - m_i||^2}{2s^2} + log\hat{P}(C_i) \\
                  &= -\frac{(x - m_i)^T(x - m_i)}{2s^2} + log\hat{P}(C_i) \\
                  &= -\frac{(x^T - 2m_i^Tx + m_i^Tm_i)}{2s^2} + log\hat{P}(C_i)
\end{align*}$

Comme $x^Tx \textrm{ commun } \forall h_i(x)$ \
$P(C_i|x) = h_i(x) = w_i^Tx + w_i^0$ 

où
$\begin{cases}
w_i = m_i \\
w_i^0 = -\frac{1}{2}||m_i||^2 - 2s^2log\hat{P}(C_i)
\end{cases}$

Décision optimale est:
$\alpha^* = 
\begin{cases}
\alpha_{K+1} & \textrm{ si } P(C_i|x) < 1 - \lambda, & \forall i = 1,..., K\\
\arg\max\limits_{\alpha_i=1}^{\alpha_K} P(C_i|x) & \textrm{ autrement }
\end{cases}$

## Q3C
Faites une implémentation du modèle présenté au point précédent en utilisant l'interface scikit-learn, permettant ainsi de l’utiliser similairement aux autres algorithmes disponibles dans la librairie. Implémentez les méthodes `fit`, `predict`, `predict_proba` et `score` dans votre modèle. Pour la fonction `score`, utilisez le coût total de l'application de votre classifieur sur les données (somme du coût des rejets et du coût des mauvais classements). Utilisez la cellule suivante pour donner votre réponse par une implémentation dans une classe Python nommée `ClassifieurAvecRejet`.

## Q3C
Make an implementation of the model presented in the previous point using the scikit-learn interface, thus allowing it to be used similarly to the other algorithms available in the library. Implement the `fit`,` predict`, `predict_proba` and` score` methods in your model. For the `score` function, use the total cost of applying your classifier to the data (sum of the cost of rejects and the cost of badly classified samples). Use the following cell to give your answer by an implementation in a Python class named `ClassifieurAvecRejet`.

### Patron de code réponse à l'exercice Q3C

### Q3C answer code template

In [ ]:
class ClassifieurAvecRejet:
    
    def __init__(self, _lambda=1):
        # _lambda est le coût de rejet
        # _lambda is the cost of reject
        self._lambda = _lambda
    

    def fit(self, X, y):
        # *** TODO Q3C ***
        # Entraînez votre modèle à partir des données fournies en argument
        # Train the model from datasets given as arguments
        pass # Retirez le pass / remove the pass
        # *****

    
    def predict_proba(self, X):
        # *** TODO Q3C ***
        # Retournez la probabilité d'appartenance à chaque classe
        # pour les données passées en argument.
        # Cette fonction peut supposer que fit() a préalablement été appelé.
        # Indice: calculez les termes de l'équation de Bayes séparément.
        # Return the class conditional probabilities to each class according
        # to the data given as argument.
        # This function can assume that fit() has been previously called.
        # Tip: compute each component of Bayes equation separately.
        pass # Retirez le pass / remove the pass
        # *****


    def predict(self, X):
        # *** TODO Q3C ***
        # Retournez les prédictions de classes pour les données
        # passées en argument.
        # Cette fonction peut supposer que fit() a préalablement été appelé.
        # Indice: vous pouvez appeler predict_proba() pour éviter une
        # redondance du code.
        # Return the class predictions for the data pass as arguments.
        # This function can assume that fit() has been previously called.
        # Tip: you can call predict_proba() to avoid repeating some code.
        pass # Retirez le pass / remove the pass
        # *****
    
    
    def score(self, X, y):
        # *** TODO Q3C ***
        # Retournez le score de performance, tenant compte des données rejetées
        # si lambda < 1.0, pour les données passées en argument.
        # Cette fonction peut supposer que fit() a préalablement été exécuté.
        # Return the performance score, taking into account rejected samples
        # if lambda < 1.0, for data given as argument.
        # This function can assume that fit() has been previously called.
        pass # Retirez le pass / remove the pass
        # *****

### Entrez votre solution à Q3C dans la cellule ci-dessous

### Enter your answer to Q3C in the cell below

## Q3D
Utilisez le jeu de données des Iris de Fisher pour tester l'algorithme que vous avez implémenté à la sous-question précédente. Pour ce faire, exécutez l'algorithme en variant le coût de rejet. Testez avec les coûts de rejet suivants : $\lambda=\{0.1, 0.3, 0.5, 1.0\}$.

Pour chaque configuration, rapportez l’erreur empirique correspondant au taux d’erreur de classement sur le jeu de données au complet (erreur sur le jeu d’entraînement). Représentez également les résultats visuellement, en traçant les données (avec indicateurs de classe), les régions de décision dans des figures 2D, incluant les régions de rejet, pour toutes les paires de variables. Fournissez votre solution dans la cellule suivante.

## Q3D
Use the Fisher Iris dataset to test the algorithm you implemented in the previous sub-question. To do this, run the algorithm by varying the cost of rejection. Test with the following rejection costs: $\lambda=\{0.1, 0.3, 0.5, 1.0\}$.

For each configuration, report the empirical error corresponding to the classification error rate on the entire dataset (error on the training set). Also represent the results visually, plotting the data (with class indicators), decision regions in 2D figures, including rejection regions, for all pairs of variables. Provide your solution in the next cell.

### Patron de code réponse à l'exercice Q3D

### Q3D answer code template

In [ ]:
# Durée d'exécution maximale / maximum execution time
TMAX_Q3D = 1.0

# Dictionnaire pour enregistrer les erreurs selon les classifieurs
# Dictionary to record classification errors
erreurs = collections.defaultdict(list)
erreurs['Classifieurs'] = []

# *** TODO Q3D ***
# Chargez jeu de données Iris dans une variable nommée 'data'
# Load dataset Iris in the variable named 'data'
# ******

# Créer une liste contenant toutes les paires possibles
# Create a list with all possible pairs
pairs = [(i, j) for i in range(4) for j in range(i+1, 4)]

# Tester le classifieur avec différents lambda pour toutes les paires
# Test the classifier with different lambda over all pairs
for (f1, f2) in pairs:
    f1_name = data.feature_names[f1]
    f2_name = data.feature_names[f2]
    
    # *** TODO Q3D ***
    # Créez un jeu de données contenant seulement les
    # mesures désignées par f1 et f2

    # Créez grille permettant d'afficher régions de décision pour chaque classifieur
    # Indices : numpy.meshgrid pourrait vous être utile, mais n'utilisez pas un pas trop petit!
    # Create a grid for displaying various decision regions for each classifier
    # Tips: numpy.meshgrid can be useful, but don't use a too small step size
    # ******

    # On initialise les classifieurs avec différents paramètres lambda
    classifieurs = [ClassifieurAvecRejet(0.1),
                    ClassifieurAvecRejet(0.3),
                    ClassifieurAvecRejet(0.5),
                    ClassifieurAvecRejet(1)]

    # Créer une figure à plusieurs sous-graphes pour pouvoir montrer,
    # pour chaque configuration, les régions de décisions, incluant
    # la zone de rejet
    # Create a figure with several sub-plots to show, for all configuration,
    # decision regions, including the reject zone
    fig, subfigs = pyplot.subplots(2, 2, sharex='all', sharey='all',
                                   tight_layout=True)
    t1 = time.time()
    for clf,subfig in zip(classifieurs, subfigs.reshape(-1)):
        clf_name = clf.__class__.__name__ + " (\u03BB={})".format(clf._lambda)        
        if clf_name not in erreurs['Classifieurs']:
            erreurs['Classifieurs'].append(clf_name)

        # *** TODO Q3D ***
        # Entraînez le classifieur
        # Train the classifier

        # Stockez la valeur de l'erreur dans la variable err
        # Store error value in variable err

        # Utilisez la grille pour afficher les régions de décision, 
        # INCLUANT LA ZONE DE REJET, de même que les points colorés selon 
        # leur vraie classe
        # Use the grid to display the decision regions, INCLUDING THE
        # REJECT ZONE, as well the points coloured according to their
        # real labels.
        # ******

        # Ajouter l'erreur pour affichage
        # Add error for displaying
        erreurs[f'{f1_name} {f2_name}'].append(err)

        # Ajouter un titre et des étiquettes d'axes
        # Add title and axis labels
        subfig.set_title("\u03BB="+str(clf._lambda))
        subfig.set_xlabel(data.feature_names[f1])
        subfig.set_ylabel(data.feature_names[f2])
        
    ### Ne pas modifier / do not modify ###
    t2 = time.time()
    duration = t2 - t1
    if duration > TMAX_Q3D:
        print(f"\x1b[31m[ATTENTION] Votre code pour la question Q3D " +
              f"met trop de temps à s'exécuter! Le temps maximum " + 
              f"permis est de {TMAX_Q3D:.4f} secondes, mais votre " +
              f"code a requis {duration:.4f} secondes! Assurez-vous " +
              f"que vous ne faites pas d'appels bloquants (par " +
              f"exemple à show()) dans cette boucle!\x1b[0m")
        
# Affichage des erreurs / error display
clfs = erreurs.pop('Classifieurs')
df = pandas.DataFrame(erreurs, index=clfs)
display.display(df)

### Entrez votre solution à Q3D dans la cellule ci-dessous

### Enter your answer to Q3D in the cell below